In [1]:
from astropy.table import Table, join, Column, vstack
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio as F
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
import json
import shutil
from desitarget.targetmask import desi_mask, obsconditions, bgs_mask
from collections import Counter

In [2]:
realn=0
fba_files = glob.glob("fba_bright_north_"+str(realn)+"/fba-*.fits")

In [3]:
tiles=[]
for i in fba_files:
    tiles.append(i.split('-')[1][-13:-5])
tiles=np.array(tiles)
tiles

array(['041812', '039964', '032754', ..., '041062', '046034', '036381'],
      dtype='<U6')

In [4]:
tile=tiles[0]
print("Testing with tile",tile)

Testing with tile 041812


In [5]:
target=F.read("targets_bright_north_"+str(realn)+".fits",columns=['TARGETID','RA','DEC','DESI_TARGET','BGS_TARGET','MWS_TARGET','SUBPRIORITY','PRIORITY_INIT','NUMOBS_INIT'])

In [6]:
assign=F.read("fba_bright_north_"+str(realn)+"/"+"fba-"+tile+".fits",
              columns=['FIBER','TARGETID','LOCATION','PETAL_LOC','TARGET_RA','TARGET_DEC','FA_TARGET','FA_TYPE'],ext="FASSIGN")
avail=F.read("fba_bright_north_"+str(realn)+"/"+"fba-"+tile+".fits",ext="FAVAIL")

In [7]:
len(assign), len(avail)

(5020, 42774)

In [8]:
assign

array([( 400, 288230378240522874,    0, 0, 128.3649244 , 73.56634431, 2305843009213693952, 1),
       ( 449, 288230378240522728,    1, 0, 128.388773  , 73.54570196, 2305843009213693952, 1),
       ( 420, 288230378240491583,    2, 0, 128.23088074, 73.56285095, 1152921504606846976, 1),
       ...,
       (4944, 288230378240508053, 9524, 9, 130.73736811, 72.30702013,         34359738368, 2),
       (4909, 288230378240536387, 9525, 9, 130.57627985, 72.28115657,          4294967296, 4),
       (4900, 288230378240507543, 9526, 9, 130.50802352, 72.28574327, 2305843017803628544, 3)],
      dtype=[('FIBER', '>i4'), ('TARGETID', '>i8'), ('LOCATION', '>i4'), ('PETAL_LOC', '>i2'), ('TARGET_RA', '>f8'), ('TARGET_DEC', '>f8'), ('FA_TARGET', '>i8'), ('FA_TYPE', 'u1')])

In [9]:
master=join(avail,assign,join_type='left',keys=["FIBER","LOCATION"])

In [10]:
master

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE
int32,int32,int64,int64,int16,float64,float64,int64,uint8
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240542339,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240542322,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240542190,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1


In [11]:
dict1=dict(zip(target["TARGETID"],target["BGS_TARGET"]))

aval_bgs = [dict1.get(i,-1) for i in master["TARGETID_1"]]
master['BGS_TARGET_1']=aval_bgs
assi_bgs = [dict1.get(i,-1) for i in master["TARGETID_2"]]
master['BGS_TARGET_2']=assi_bgs

In [12]:
master

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE,BGS_TARGET_1,BGS_TARGET_2
int32,int32,int64,int64,int16,float64,float64,int64,uint8,int64,int64
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,2,1
311,0,288230378240542339,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1
311,0,288230378240542322,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,1,1
311,0,288230378240542190,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1


In [13]:
is_availableintarget = np.zeros(len(master), dtype=bool)
id_availableintarget = np.isin(master['TARGETID_1'], target["TARGETID"])
is_availableintarget[id_availableintarget] = True
master['AVAIL_IN_TARGET'] = is_availableintarget

In [14]:
master

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE,BGS_TARGET_1,BGS_TARGET_2,AVAIL_IN_TARGET
int32,int32,int64,int64,int16,float64,float64,int64,uint8,int64,int64,bool
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,2,1,True
311,0,288230378240542339,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1,False
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240542322,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1,False
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,1,1,True
311,0,288230378240542190,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1,False
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True


In [15]:
master_in=master[master["AVAIL_IN_TARGET"]==True]
len(master_in)

32637

In [16]:
np.unique(master_in[master_in["BGS_TARGET_2"]==-1]["FA_TYPE","BGS_TARGET_2"],return_counts=True)

(array([(0, -1), (4, -1)],
       dtype=[('FA_TYPE', 'u1'), ('BGS_TARGET_2', '<i8')]),
 array([ 154, 2468]))

In [17]:
isbright_1=master_in["BGS_TARGET_1"]==2
isbright_2=master_in["BGS_TARGET_2"]==2
isfaint_1=master_in["BGS_TARGET_1"]==1
isfaint_2=master_in["BGS_TARGET_2"]==1
isfainthip_1=master_in["BGS_TARGET_1"]==9
isfainthip_2=master_in["BGS_TARGET_2"]==9
isstar_1=master_in["BGS_TARGET_1"]==0
isstar_2=master_in["BGS_TARGET_2"]==0
issky=master_in["FA_TYPE"]==4
isstd=(master_in["FA_TYPE"]==3)|(master_in["FA_TYPE"]==2)
isscience=(master_in["FA_TYPE"]==1)|(master_in["FA_TYPE"]==3)
iszero=(master_in["FA_TYPE"]==0)

In [18]:
print("NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isbright_1&isbright_2]["FIBER"])))
print("NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND FAINT ASSIGNED =",len(np.unique(master_in[isbright_1&isfaint_2]["FIBER"])))
print("NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND FAINT_HIP ASSIGNED =",len(np.unique(master_in[isbright_1&isfainthip_2]["FIBER"])))
print("NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND STAR ASSIGNED =",len(np.unique(master_in[isbright_1&isstar_2]["FIBER"])))

NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND BRIGHT ASSIGNED = 531
NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND FAINT ASSIGNED = 617
NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND FAINT_HIP ASSIGNED = 41
NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND STAR ASSIGNED = 1617


In [19]:
print("NUMBER OF FIBRES WITH FAINT AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isfaint_1&isbright_2]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT AVAILABLE AND FAINT ASSIGNED =",len(np.unique(master_in[isfaint_1&isfaint_2]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT AVAILABLE AND FAINT_HIP ASSIGNED =",len(np.unique(master_in[isfaint_1&isfainthip_2]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT AVAILABLE AND STAR ASSIGNED =",len(np.unique(master_in[isfaint_1&isstar_2]["FIBER"])))

NUMBER OF FIBRES WITH FAINT AVAILABLE AND BRIGHT ASSIGNED = 363
NUMBER OF FIBRES WITH FAINT AVAILABLE AND FAINT ASSIGNED = 833
NUMBER OF FIBRES WITH FAINT AVAILABLE AND FAINT_HIP ASSIGNED = 24
NUMBER OF FIBRES WITH FAINT AVAILABLE AND STAR ASSIGNED = 1329


In [20]:
print("NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isfainthip_1&isbright_2]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND FAINT ASSIGNED =",len(np.unique(master_in[isfainthip_1&isfaint_2]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND FAINT_HIP ASSIGNED =",len(np.unique(master_in[isfainthip_1&isfainthip_2]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND STAR ASSIGNED =",len(np.unique(master_in[isfainthip_1&isstar_2]["FIBER"])))

NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND BRIGHT ASSIGNED = 60
NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND FAINT ASSIGNED = 87
NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND FAINT_HIP ASSIGNED = 48
NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND STAR ASSIGNED = 231


In [21]:
print("NUMBER OF FIBRES WITH STAR AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isstar_1&isbright_2]["FIBER"])))
print("NUMBER OF FIBRES WITH STAR AVAILABLE AND FAINT ASSIGNED =",len(np.unique(master_in[isstar_1&isfaint_2]["FIBER"])))
print("NUMBER OF FIBRES WITH STAR AVAILABLE AND FAINT_HIP ASSIGNED =",len(np.unique(master_in[isstar_1&isfainthip_2]["FIBER"])))
print("NUMBER OF FIBRES WITH STAR AVAILABLE AND STAR ASSIGNED =",len(np.unique(master_in[isstar_1&isstar_2]["FIBER"])))

NUMBER OF FIBRES WITH STAR AVAILABLE AND BRIGHT ASSIGNED = 524
NUMBER OF FIBRES WITH STAR AVAILABLE AND FAINT ASSIGNED = 818
NUMBER OF FIBRES WITH STAR AVAILABLE AND FAINT_HIP ASSIGNED = 47
NUMBER OF FIBRES WITH STAR AVAILABLE AND STAR ASSIGNED = 2939


In [22]:
print("NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isbright_1&issky]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isfaint_1&issky]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isfainthip_1&issky]["FIBER"])))
print("NUMBER OF FIBRES WITH STAR AVAILABLE AND BRIGHT ASSIGNED =",len(np.unique(master_in[isstar_1&issky]["FIBER"])))

NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND BRIGHT ASSIGNED = 314
NUMBER OF FIBRES WITH FAINT AVAILABLE AND BRIGHT ASSIGNED = 265
NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND BRIGHT ASSIGNED = 39
NUMBER OF FIBRES WITH STAR AVAILABLE AND BRIGHT ASSIGNED = 584


In [51]:
print("NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND SKY ASSIGNED =",len(np.unique(master_in[isbright_1&issky]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT AVAILABLE AND SKY ASSIGNED =",len(np.unique(master_in[isfaint_1&issky]["FIBER"])))
print("NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND SKY ASSIGNED =",len(np.unique(master_in[isfainthip_1&issky]["FIBER"])))
print("NUMBER OF FIBRES WITH ZERO AVAILABLE AND SKY ASSIGNED =",len(np.unique(master_in[isstar_1&issky]["FIBER"])))

NUMBER OF FIBRES WITH BRIGHT AVAILABLE AND SKY ASSIGNED = 314
NUMBER OF FIBRES WITH FAINT AVAILABLE AND SKY ASSIGNED = 265
NUMBER OF FIBRES WITH FAINT_HIP AVAILABLE AND SKY ASSIGNED = 39
NUMBER OF FIBRES WITH ZERO AVAILABLE AND SKY ASSIGNED = 584


In [24]:
Count_fibers=np.ones(16)
OBS=["BRIGHT2BRIGHT","BRIGHT2FAINT","BRIGHT2FAINTHIP","BRIGHT2STAR","FAINT2BRIGHT","FAINT2FAINT","FAINT2FAINTHIP","FAINT2STAR",
    "FAINTHIP2BRIGHT","FAINTHIP2FAINT","FAINTHIP2FAINTHIP","FAINTHIP2STAR","STAR2BRIGHT","STAR2FAINT","STAR2FAINTHIP","STAR2STAR"]

Count_fibers[0]=len(np.unique(master_in[isbright_1&isbright_2]["FIBER"]))
Count_fibers[1]=len(np.unique(master_in[isbright_1&isfaint_2]["FIBER"]))
Count_fibers[2]=len(np.unique(master_in[isbright_1&isfainthip_2]["FIBER"]))
Count_fibers[3]=len(np.unique(master_in[isbright_1&isstar_2]["FIBER"]))
Count_fibers[4]=len(np.unique(master_in[isfaint_1&isbright_2]["FIBER"]))
Count_fibers[5]=len(np.unique(master_in[isfaint_1&isfaint_2]["FIBER"]))
Count_fibers[6]=len(np.unique(master_in[isfaint_1&isfainthip_2]["FIBER"]))
Count_fibers[7]=len(np.unique(master_in[isfaint_1&isstar_2]["FIBER"]))
Count_fibers[8]=len(np.unique(master_in[isfainthip_1&isbright_2]["FIBER"]))
Count_fibers[9]=len(np.unique(master_in[isfainthip_1&isfaint_2]["FIBER"]))
Count_fibers[10]=len(np.unique(master_in[isfainthip_1&isfainthip_2]["FIBER"]))
Count_fibers[11]=len(np.unique(master_in[isfainthip_1&isstar_2]["FIBER"]))
Count_fibers[12]=len(np.unique(master_in[isstar_1&isbright_2]["FIBER"]))
Count_fibers[13]=len(np.unique(master_in[isstar_1&isfaint_2]["FIBER"]))
Count_fibers[14]=len(np.unique(master_in[isstar_1&isfainthip_2]["FIBER"]))
Count_fibers[15]=len(np.unique(master_in[isstar_1&isstar_2]["FIBER"]))

In [25]:
Table([OBS,Count_fibers])

col0,col1
str17,float64
BRIGHT2BRIGHT,531.0
BRIGHT2FAINT,617.0
BRIGHT2FAINTHIP,41.0
BRIGHT2STAR,1617.0
FAINT2BRIGHT,363.0
FAINT2FAINT,833.0
FAINT2FAINTHIP,24.0
FAINT2STAR,1329.0
FAINTHIP2BRIGHT,60.0
